In [117]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import graspy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [118]:
def generate_latent_positions(n=50, d=2, acorn=None):
    """
    A function to generate an adjacency matrix.
    
    Input
    n - int
        If P is None then n is the number of latent positions to randomly generate.
    d - int
        If P is None the d is the dimension of the latent positions.
    acorn - int
        Random seed.
        
    Return
    X - np.array (shape=(n,d))
        An array of uniformly distributed points in the positive unit sphere
    """
    
    if acorn is not None:
        np.random.seed(acorn)
        
    mvns = np.random.multivariate_normal(np.zeros(d), np.eye(d), size=n)

    mvns = abs(mvns) / np.linalg.norm(mvns, axis=1)[:, None]

    unis = np.random.uniform(size=n)**(1/d)
    X = mvns * unis[:, None]
                
    return X
        
    
def identity(X, ind):
    _, d = X.shape
    return np.eye(d)


def generate_distance_matrix(A, embedding_functions, covariance_functions, ind=0,acorn=None):
    """
    A function to generate a distance matrix given an adjacency matrix. The ith column of the
    distance matrix is the Euclidean distance from the first node in the graph to the other
    nodes in the ith embedding.
    
    Input
    A - list of np.arrays (length=J) or np.array (shape=(n,d[j]))
        Latent positions or adjacency matrix.
    emebedding_functions - 
    
    covariance_functions - 
    covariances - list of np.arrays (length=J, covariances[j].shape=(d[j],d[j]))
        List of covariances to calculate the Mahalonobis distance between vectors in X.
    acorn - int
        Random seed.
        
    Return
    dist_matrix - np.array (shape=(n, J))
        Distance matrix where the ith column is the Euclidean distance from the first node in the graph
        to all of the other nodes in the graph in the ith embedding.
    """
    
    if acorn is not None:
        np.random.seed(acorn)
        
    n = A.shape[0]
        
    J = len(covariance_functions)
            
    dist_matrix = np.zeros((n, J))
    
    for j, embed in enumerate(embedding_functions):
        temp_X = embed(A)
            
        if isinstance(covariance_functions[j], np.ndarray):
            temp_cov = covariance_functions[j]
        else:
            temp_cov = covariance_functions[j](temp_X, ind)
            
        diffs = np.array([temp_X[ind] - temp_X[i] for i in range(n)])
        dist_matrix[:, j] = np.sqrt(np.array([diffs[i].T @ temp_cov @ diffs[i] for i in range(n)]))
       
        if np.sum(dist_matrix[:, j] < 0) > 0:
            print("i broke on distance %i"%(j))
    return dist_matrix  


def generate_S_indices(dist_matrix, alpha, n_inds=1):
    """
    A function to generate the nodes of interest.
    
    Input
    dist_matrix - np.array (shape=(n,J))
        Array containing the distances between the vertex of interest and the other n - 1
        vertices. It is assumed that the vertex of interest is indexed by 0.
    alpha - float or array-like
        Coefficients of the distances to generate the ground truth. alpha can only be an int
        if J == 2. If alpha is array-like then the sum of alpha must be 1.
    n_inds - int or func
        Number of vertices in the vertex set of interest. If n_inds is a function then the
        ceiling of n_inds(n) is used.
        
    Return
    S - np.array (length=n_inds)
        A list of indices of length n_inds in range(1,n) corresponding to vertices of interest.
    """
    
    n, J = dist_matrix.shape
    
    if isinstance(alpha, float):
        assert J == 2
        alpha = [alpha, 1-alpha]
    
    assert np.sum(alpha) == 1
    
    if not isinstance(n_inds, int):
        n_inds = int(np.math.ceil(n_inds(n)))
    
    new_distances = np.average(dist_matrix, axis=1, weights=alpha)
    
    new_nearest_neighbors = np.argsort(new_distances)
    
    S = new_nearest_neighbors[1:n_inds+1]
    
    return S


def optimal_distance(dist_matrix, S_indices, model_name=None, return_new_dists=True):
    """
    A function to find the weights of optimal linear combination of distances.
    
    Input
    dist_matrix - np.array (shape=(n, J))
        Array containing the distances between the vertex of interest and the other n - 1
        vertices. It is assumed that the vertex of interest is indexed by 0.
    S_indices - array-like
        Array-like containing the indices of the vertices that should be at the top of the
        nomination list for the vertex of interest.
        
    Return
    weights - np.array (length=J)
        Array containing the coefficients for the optimal distance function.
    """
    
    n, J = dist_matrix.shape
    M = np.sum(abs(dist_matrix))
    
    S = len(S_indices)
    Q_indices = np.array([i for i in range(1, n) if i not in S_indices])
    Q = len(Q_indices)
    
    M = np.sum(abs(dist_matrix))
    
    if model_name is not None:
        m = gp.Model(name='%s'%(model_name))
    else:
        m= gp.Model()
        
    m.setParam('OutputFlag', 0)

    ind = m.addVars(Q, vtype=GRB.BINARY, name='ind')
    m.setObjective(gp.quicksum(ind), GRB.MINIMIZE)

    w = m.addVars(J, lb=0, ub=1, vtype=GRB.CONTINUOUS, name='w')
    m.addConstr(w.sum() == 1)

    # There's likely a more pythonic way to set up these constraints (in particular using m.addConstrs(..))
    for s in S_indices:
        temp_s = gp.tupledict([((i), dist_matrix[s, i]) for i in range(J)])
        for i, q in enumerate(Q_indices):
            temp_q = gp.tupledict([((i), dist_matrix[q, i]) for i in range(J)])
            m.addConstr(w.prod(temp_s) <= w.prod(temp_q) + ind[i]*M)
        
    m.optimize()
    
    alpha_hat = np.array([i.X for i in list(w.values())])
    
    if model_name:
        m.write('%s.ip'%(model_name))
        
    if return_new_dists:
        return alpha_hat, np.average(dist_matrix, axis=1, weights=alpha_hat)
    
    return alpha_hat

In [119]:
np.random.seed(11)

# Number of vertices, latent space dimension
n, d = 50, 2

# Latent positions
X = generate_latent_positions(n=n, d=d)

# Probability matrix
P = X @ X.T

# Embedding functions 
ASE = graspy.embed.AdjacencySpectralEmbed(n_components=d).fit_transform
LSE = graspy.embed.LaplacianSpectralEmbed(n_components=d).fit_transform
embedding_functions = [ASE, LSE]

# Covariance functions used to calculate distances 
ASE_cov = identity
LSE_cov = identity
covariance_functions = [ASE_cov, LSE_cov]

# Vertex of interest
v_star = 0

# Distance matrix from vertex of interest to all other vertices (including itself)
dist_matrix = generate_distance_matrix(P, embedding_functions, covariance_functions, v_star)

# Define "true" distance
alpha = 0.1

# Find the 6 closest vertices to v star as defined by the "true" distance
S_indices = generate_S_indices(dist_matrix, alpha, n_inds=6)

# Use the first five to learn the appropriate distance, the last to evaluate
S_indices, s_star = S_indices[:-1], S_indices[-1]

# Find weights, new distances 
alpha_hat, new_distance = optimal_distance(dist_matrix, S_indices)

print("True alpha:", [alpha, 1-alpha])
print("Estimated alpha:", alpha_hat)

# The 5 things closest to v star in the new distance are the correct vertices
print("\n")
print("S indices:", S_indices)
print("Five closest vertices to v star according to learned distance:", np.argsort(new_distance)[1:6])

# s_star is the second closest vertex to v star in the new distance
print("\n")
print("s star:", s_star)
print("Next two closest vertices to v star according to learned distance:", np.argsort(new_distance)[6:8])

print("\n")
print("Seven closest vertices to v star according to ASE:", np.argsort(dist_matrix[:, 0])[1:8])
print("Seven closest vertices to v star according to LSE:", np.argsort(dist_matrix[:, 1])[1:8])

True alpha: [0.1, 0.9]
Estimated alpha: [0.08201382 0.91798618]


S indices: [10 12 26 27 49]
Five closest vertices to v star according to learned distance: [10 12 27 49 26]


s star: 34
Next two closest vertices to v star according to learned distance: [34 45]


Seven closest vertices to v star according to ASE: [10 26 12 27 36 45  3]
Seven closest vertices to v star according to LSE: [10 12 49 34 27 31 26]
